### this notebook is going to be used for merging the two datasets

In [3]:
#import fastai
#from fastai.tabular import *
#from fastai.text import *

import pandas as pd
import numpy as np

In [20]:
#first reload the bch dataset
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'

bch_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/bch'
egh_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/egh'
bch_df = pd.read_csv(bch_data_path +'/complete_clean_bch_data.csv', index_col = 0, low_memory = False)
egh_df = pd.read_csv(egh_data_path +'/complete_clean_egh_data.csv', index_col = 0, low_memory = False)


In [21]:
#now reorder the EGH dataset columns to match the BCH dataset columns
egh_df = egh_df.reindex(columns=bch_df.columns)

In [22]:
#now add a column to each indicating the site
egh_df['site'] = 'EGH'
bch_df['site'] = 'BCH'

In [23]:
#now merge them
combo_df = pd.concat([bch_df, egh_df], axis = 0)
combo_df.shape

(165957, 59)

In [24]:
combo_df.dropna(subset = ['CleanSubjectiveNotes','CleanMedicalHistory'], inplace = True)
combo_df.shape

(165433, 59)

In [25]:
#need to reindex because there's some overlap in the id's from the egh and bch datasets

new_index = np.arange(len(combo_df))

combo_df.index = new_index

In [ ]:
dx_code_dict = {code:i for i,code in enumerate(set(combo_df['MainDiagnosisCode']))}

def convert_dxcode(s):
    code = dx_code_dict[s]
    return (code)

combo_df['recoded_diagnosis'] = combo_df['MainDiagnosisCode'].map(convert_dxcode)

In [26]:
#need to create a set of validation indices to use across all databunches...
import random

valid_idxs = random.sample(list(combo_df.index), len(combo_df)//5)

valid_df = pd.DataFrame(valid_idxs)
valid_df.to_csv(path + '/valid_idxs.csv', index=False)

In [29]:
# make the date columns
date_columns = ['Triage Date & Time', 'Reg Date & Time']
for col in date_columns:
    add_datepart(combo_df, col, drop = False, time = True)
    add_cyclic_datepart(combo_df, col, time = True)

In [ ]:
clin['ICUvsother'].fillna(0, inplace = True)

### gonna make balanced combo datasets

In [31]:
Counter(combo_df.target), Counter(combo_df.target3)

(Counter({1.0: 149218, 3.0: 2169, 2.0: 13701, 4.0: 345}),
 Counter({1.0: 149218, 0.0: 16215}))

In [32]:
admit_df = combo_df[combo_df.target3 == 0]

dc_df = combo_df[combo_df.target3==1]
dc_df = dc_df.sample(n= len(admit_df))  #this is the number of admissions from target 3



In [33]:
combo_balanced_df1 = pd.concat([admit_df, dc_df], axis = 0)

In [34]:
icu_df = combo_df[combo_df.target2 == 4]
madmit_df = combo_df[combo_df.target2 ==3].sample(n = len(icu_df))
sadmit_df = combo_df[combo_df.target2 ==2].sample(n = len(icu_df))
disc_df = combo_df[combo_df.target2 == 1].sample(n = len(icu_df))

In [35]:
#this is a dataframe with equal numbers of all 4 classes
combo_balanced_df2 = pd.concat([icu_df, madmit_df, sadmit_df, disc_df], axis = 0)

In [36]:
non_icu_df = combo_df[combo_df.target2 !=4].sample(n = len(icu_df))

In [37]:
combo_balanced_df3 = pd.concat([icu_df, non_icu_df], axis = 0)

In [39]:

combo_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

combo_balanced_df1.to_csv(combo_data_path + '/balanced_admit_dc_combo_data.csv')

combo_balanced_df2.to_csv(combo_data_path +'/balanced_4cls_combo_data.csv')

combo_balanced_df3.to_csv(combo_data_path +'/balanced_icuvsother_combo_data.csv')

combo_df.to_csv(combo_data_path +'/complete_clean_combo_data.csv')



### this section is for only the NLP related fields of the dataset

In [4]:
#first reload the bch dataset
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'

bch_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/bch'
egh_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/egh'
bch_df = pd.read_csv(bch_data_path +'/clean_nlp_data_bch.csv', index_col = 0, low_memory = False)
egh_df = pd.read_csv(egh_data_path +'/clean_nlp_data_egh.csv', index_col = 0, low_memory = False)


In [5]:
#now reorder the EGH dataset columns to match the BCH dataset columns
egh_df = egh_df.reindex(columns=bch_df.columns)

In [6]:
#now add a column to each indicating the site
egh_df['site'] = 'EGH'
bch_df['site'] = 'BCH'

In [7]:
#now merge them
combo_df = pd.concat([bch_df, egh_df], axis = 0)
combo_df.shape

(165957, 14)

In [9]:
combo_df.head()

,CleanSubjectiveNotes,pmhx,num_comorbids,outcome,target,service,target2,discharge,target3,dispo,target4,ICUvsother,target5,site
148,patient states that she feels shaky patient de...,no significant medical history,0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0,BCH
156,NaN,substance misuse,1,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0,BCH
197,patient states no pain,no significant medical history,0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0,BCH
217,denied chest pain,no significant medical history,0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0,BCH
218,denies pain eating emergency department lunch ...,"dialysis haemo, type one diabetes, chronic ren...",5,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0,BCH


In [10]:
combo_df.dropna(subset = ['CleanSubjectiveNotes','pmhx'], inplace = True)
combo_df.shape

(165268, 14)

In [11]:
#need to reindex because there's some overlap in the id's from the egh and bch datasets

new_index = np.arange(len(combo_df))

combo_df.index = new_index

In [12]:

combo_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

combo_df.to_csv(combo_data_path + '/nlp_combo_data.csv')



In [13]:
combo_df.shape

(165268, 14)

In [14]:
combo_df.columns

Index(['CleanSubjectiveNotes', 'pmhx', 'num_comorbids', 'outcome', 'target',
       'service', 'target2', 'discharge', 'target3', 'dispo', 'target4',
       'ICUvsother', 'target5', 'site'],
      dtype='object')

In [19]:
len(combo_df), len(egh_df), len(bch_df)

(165268, 63465, 102492)